In [1]:
# To find out where the pyspark
import findspark
findspark.init()

In [2]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext("local", "first app")

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [48]:
import matplotlib.pyplot as plt
import pandas as pd

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import MulticlassClassificationEvaluator # 0 to 9 multi classification
from pyspark.ml.evaluation import BinaryClassificationEvaluator # 0 or 1 binary classification

In [5]:
df_train = (spark.read.options(header = False, inferSchema = True).csv("./mnist_train.csv"))

In [6]:
df_train.count()

60000

In [7]:
feature_culumns = ["_c" + str(i+1) for i in range(784)]

## train, test 데이터 dataframe화 

In [9]:
vectorizer = VectorAssembler(inputCols=feature_culumns, outputCol="features")
train = (vectorizer.transform(df_train).select("_c0", "features").toDF("label", "features").cache())

In [10]:
df_test = (spark.read.options(header = False, inferSchema = True).csv("./mnist_test.csv"))
test = (vectorizer.transform(df_test).select("_c0", "features").toDF("label", "features").cache())

In [11]:
df_test.count()

10000

## spark ml의 logistic regression사용

In [15]:
lr = LogisticRegression(featuresCol="features", 
                        labelCol="label", 
                        regParam=0.1, 
                        maxIter=5)

## 0 to 9 multi classification 정확도 예측

In [16]:
lr_model = lr.fit(train)

In [17]:
test_pred = lr_model.transform(test)

In [18]:
score = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [19]:
score.evaluate(test_pred)

0.8595

## 0 or 1 binary classification 정확도 예측

In [29]:
a = train.filter(train.label==0)
b = train.filter(train.label==1)

In [38]:
binary_train = a.union(b)

In [40]:
aa = test.filter(test.label==0)
bb = test.filter(test.label==1)

In [41]:
binary_test = aa.union(bb)

In [42]:
binary_model = lr.fit(binary_train)

In [43]:
test_binary_pred = binary_model.transform(binary_test)

In [46]:
binary_evaluator = BinaryClassificationEvaluator()

In [47]:
binary_evaluator.evaluateuate(test_binary_pred)

0.9999955048098534